In [1]:
import os
import random
from io import open
import unicodedata
import string
import re

import torch
import torchaudio
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from pathlib import Path
import kaldi_io
import math
import torch.utils.data as Data


%matplotlib inline


In [4]:
from lib.Data_show import Data_show
from lib.Phone_cla_Dataset import Phone_cla_Dataset
from lib.Decoder import Decoder

In [6]:
Data_show().phone2class

{1: 0, 3: 0, 129: 1, 63: 2, 61: 3, 27: 4, 128: 5, 64: 6, 92: 7, 69: 8}

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]="3"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 加载数据

In [2]:
phone_label = { u:d for u,d in kaldi_io.read_vec_int_ark("feats/ali_chain.1.ph") }
feats = { u:d for u,d in kaldi_io.read_mat_scp("../wake_up_align_sil_shared_recorp/train_fbank/feats.scp") }

In [5]:
print(len(feats))
print(len(phone_label))

96489
96489


In [7]:
list(feats.keys())[-30010:-30000]

['SV0270_6_12_N2808',
 'SV0270_6_12_N2809',
 'SV0270_6_12_N2810',
 'SV0270_6_12_N2812',
 'SV0270_6_12_N2814',
 'SV0270_6_12_N2816',
 'SV0270_6_12_N2863',
 'SV0270_6_12_N2867',
 'SV0270_6_12_N2872',
 'SV0270_6_12_N2876']

## 自定义数据集

In [8]:
utt="SV0287_6_15_N3046"
a=np.zeros(feats[utt].shape[0], int)
for i in range(a.shape[0]):
    print("i:\t", i, "\t",(i+1)//3)
    a[i]=phone_label[utt][(i)//3]

i:	 0 	 0
i:	 1 	 0
i:	 2 	 1
i:	 3 	 1
i:	 4 	 1
i:	 5 	 2
i:	 6 	 2
i:	 7 	 2
i:	 8 	 3
i:	 9 	 3
i:	 10 	 3
i:	 11 	 4
i:	 12 	 4
i:	 13 	 4
i:	 14 	 5
i:	 15 	 5
i:	 16 	 5
i:	 17 	 6
i:	 18 	 6
i:	 19 	 6
i:	 20 	 7
i:	 21 	 7
i:	 22 	 7
i:	 23 	 8
i:	 24 	 8
i:	 25 	 8
i:	 26 	 9
i:	 27 	 9
i:	 28 	 9
i:	 29 	 10
i:	 30 	 10
i:	 31 	 10
i:	 32 	 11
i:	 33 	 11
i:	 34 	 11
i:	 35 	 12
i:	 36 	 12
i:	 37 	 12
i:	 38 	 13
i:	 39 	 13
i:	 40 	 13
i:	 41 	 14
i:	 42 	 14
i:	 43 	 14
i:	 44 	 15
i:	 45 	 15
i:	 46 	 15
i:	 47 	 16
i:	 48 	 16
i:	 49 	 16
i:	 50 	 17
i:	 51 	 17
i:	 52 	 17
i:	 53 	 18
i:	 54 	 18
i:	 55 	 18
i:	 56 	 19
i:	 57 	 19
i:	 58 	 19
i:	 59 	 20
i:	 60 	 20
i:	 61 	 20
i:	 62 	 21
i:	 63 	 21
i:	 64 	 21
i:	 65 	 22
i:	 66 	 22
i:	 67 	 22
i:	 68 	 23
i:	 69 	 23
i:	 70 	 23
i:	 71 	 24
i:	 72 	 24
i:	 73 	 24
i:	 74 	 25
i:	 75 	 25
i:	 76 	 25
i:	 77 	 26
i:	 78 	 26
i:	 79 	 26
i:	 80 	 27
i:	 81 	 27
i:	 82 	 27
i:	 83 	 28
i:	 84 	 28
i:	 85 	 28
i:	 86 

In [25]:
max(list(Data_show().phone2class.values()))

8

In [10]:
def function_A(x):
    if x in Data_show.phone2class:
        result = Data_show.phone2class[x]
    else:
        result = 9
        
    return result

function_vector = np.vectorize(function_A)

In [11]:
function_vector(phone_label[utt])

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 3, 3, 3, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 5, 5, 6, 6, 7, 7, 8, 8, 8, 8, 8,
       8, 8, 0, 0])

In [5]:
data_set_train = Phone_cla_Dataset(phone_label, feats)

In [19]:
len(Data_show().phone2class)

10

In [6]:
data_set_train.phone_label_nd[20:51]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9])

In [17]:
data_set_dev=Phone_cla_Dataset(phone_label_dev, feats_dev)

In [54]:
print(data_set_train.feats_nd.shape)
print(data_set_train.phone_label_nd.shape)
print(data_set_train.phone_label_nd[14440478])

(23377008, 40)
(23377008,)
9


In [39]:
print(train_data.shape)
print(train_label.shape)

torch.Size([23377008, 40])
torch.Size([23377008])


In [38]:
train_data = torch.Tensor(data_set_train.feats_nd).to(device)
train_label = torch.LongTensor(data_set_train.phone_label_nd).to(device)
# test_data = torch.Tensor(data_set_dev.feats_nd).to(device)
# test_label = torch.LongTensor(data_set_dev.phone_label_nd).to(device)

# 模型搭建

In [40]:
class DNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 128)
        self.fc4 = nn.Linear(128, 128)
        self.fc5 = nn.Linear(128, num_classes)


    def forward(self, input):
        x = torch.sigmoid(self.fc1(input))
        x = torch.sigmoid(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        x = self.fc5(x)
        
        return x

## 训练迭代

In [41]:
LEARNING_RATE = 0.001  #0.001
EPOCH = 10        #400 best
BATCH_SIZE = 150
input_size=40
num_classes=10

In [42]:
print(train_data.shape)
print(train_label.shape)

torch.Size([23377008, 40])
torch.Size([23377008])


In [43]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# prepare the data loader
training_set = Data.TensorDataset(train_data,
                                  train_label)
training_loader = Data.DataLoader(dataset=training_set,
                                      batch_size=BATCH_SIZE,
                                      shuffle=True)
# testing_set = Data.TensorDataset(test_data,
#                                  test_label)
# testing_loader = Data.DataLoader(dataset=testing_set,
#                                      batch_size=BATCH_SIZE,
#                                      shuffle=False)
model = DNN(input_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
for epoch in range(EPOCH):
    correct_train = 0
    total_train = 0
    
    total_loss = 0
    ite = 0
    print_every = 2000
    for (data, label) in training_loader:
        ite +=1
        data = data
        label = label
        pred_label = model(data)
        loss = criterion(pred_label, label)
        optim.zero_grad()
        loss.backward()
        total_loss += loss.item()
        optim.step()
        _, answer = torch.max(pred_label.data, 1)
        total_train += label.size(0)
        correct_train += (answer == label).sum()
        if ite % print_every == 0:
            print("total_loss:",total_loss/print_every, "\tloss:",loss.item())
            total_loss=0
    print('Epoch {:3d} Accuracy on training data: {}% ({}/{})'
          .format(epoch, (100 * correct_train / total_train), correct_train, total_train))
    # pytorch 0.4 feature, not calculate grad on test set
#     with torch.no_grad():
#         correct_test = 0
#         total_test = 0
#         for (data, label) in testing_loader:
#             pred_label = model(data)
#             _, answer = torch.max(pred_label.data, 1)
#             total_test += label.size(0)
#             correct_test += (answer == label).sum()
#         print('          Accuracy on testing data: {}% ({}/{})'
#               .format((100 * correct_test / total_test), correct_test, total_test))


total_loss: 0.9017950873970986 	loss: 0.7568432688713074
total_loss: 0.7083490777760744 	loss: 0.7602202296257019
total_loss: 0.6546489250510931 	loss: 0.6590193510055542
total_loss: 0.6343577071577311 	loss: 0.5998808741569519
total_loss: 0.626841609030962 	loss: 0.695142388343811
total_loss: 0.6147419776022435 	loss: 0.41271841526031494
total_loss: 0.6048995875567198 	loss: 0.621376097202301
total_loss: 0.5992956459671259 	loss: 0.6758029460906982
total_loss: 0.5889536770284176 	loss: 0.7117871642112732
total_loss: 0.5784980691671372 	loss: 0.5600869059562683
total_loss: 0.5794701857566833 	loss: 0.5108597874641418
total_loss: 0.5741884642839432 	loss: 0.6101375818252563
total_loss: 0.5710383405983448 	loss: 0.6082461476325989
total_loss: 0.566199341237545 	loss: 0.5543668866157532
total_loss: 0.5651721824109555 	loss: 0.6631925106048584
total_loss: 0.5647353216856718 	loss: 0.6040521860122681
total_loss: 0.5602927262485027 	loss: 0.5938203930854797
total_loss: 0.5622905475497246 	lo

total_loss: 0.5057706053256988 	loss: 0.3555677831172943
total_loss: 0.5038239756375551 	loss: 0.3999011516571045
total_loss: 0.5053607027232647 	loss: 0.5719572901725769
total_loss: 0.5020421640723943 	loss: 0.6426869034767151
total_loss: 0.5034541816413403 	loss: 0.5313351154327393
total_loss: 0.5071043657064438 	loss: 0.3906393051147461
total_loss: 0.5042432020008564 	loss: 0.5003362894058228
total_loss: 0.5047712115347386 	loss: 0.6003991961479187
total_loss: 0.5039901158362627 	loss: 0.4377637207508087
total_loss: 0.5025855068266392 	loss: 0.5014123320579529
total_loss: 0.5009740086197854 	loss: 0.33790668845176697
Epoch   1 Accuracy on training data: 83% (19471247/23377008)
total_loss: 0.5027870022654534 	loss: 0.5765292644500732
total_loss: 0.5020775401890278 	loss: 0.5267022848129272
total_loss: 0.5041595921367407 	loss: 0.4300031065940857
total_loss: 0.5020541747510433 	loss: 0.5202156901359558
total_loss: 0.5012483578920365 	loss: 0.5916978716850281
total_loss: 0.500258799925

total_loss: 0.49391406621038914 	loss: 0.6468468308448792
total_loss: 0.49390364438295364 	loss: 0.399790495634079
total_loss: 0.4955712441951036 	loss: 0.43777501583099365
total_loss: 0.494790987983346 	loss: 0.6173843741416931
total_loss: 0.49426760330796243 	loss: 0.5941482186317444
total_loss: 0.48993456445634365 	loss: 0.4446290135383606
total_loss: 0.49273783057928083 	loss: 0.36977219581604004
total_loss: 0.49478620103001597 	loss: 0.4739633798599243
total_loss: 0.4943107169419527 	loss: 0.4799841344356537
total_loss: 0.496658875644207 	loss: 0.677990734577179
total_loss: 0.4940006350129843 	loss: 0.5584549903869629
total_loss: 0.4950495003312826 	loss: 0.4939796030521393
total_loss: 0.494631915345788 	loss: 0.44339579343795776
total_loss: 0.4951487011760473 	loss: 0.5773195624351501
total_loss: 0.49317343451082707 	loss: 0.6233339905738831
total_loss: 0.49504155430197716 	loss: 0.4494735598564148
total_loss: 0.49368979734182356 	loss: 0.34746867418289185
total_loss: 0.493857403

total_loss: 0.4840093150883913 	loss: 0.46074849367141724
total_loss: 0.4880372893810272 	loss: 0.4382070004940033
total_loss: 0.49047561779618265 	loss: 0.4439997971057892
total_loss: 0.4889444535151124 	loss: 0.5763755440711975
total_loss: 0.48774590146541597 	loss: 0.5218497514724731
total_loss: 0.4878034530133009 	loss: 0.7144261598587036
total_loss: 0.48672918225079775 	loss: 0.5696423053741455
total_loss: 0.48675281503796575 	loss: 0.45311659574508667
total_loss: 0.486777396529913 	loss: 0.44144102931022644
total_loss: 0.48657957795262335 	loss: 0.5384525656700134
total_loss: 0.4862819849252701 	loss: 0.42170554399490356
total_loss: 0.48560625536739827 	loss: 0.6800816059112549
total_loss: 0.4884903526753187 	loss: 0.3635596036911011
total_loss: 0.4884101554900408 	loss: 0.4914674460887909
total_loss: 0.48767554002255203 	loss: 0.5678644776344299
total_loss: 0.48747782999277117 	loss: 0.43950939178466797
total_loss: 0.4836276081055403 	loss: 0.43891942501068115
total_loss: 0.4861

total_loss: 0.48253033083677294 	loss: 0.48177745938301086
total_loss: 0.4843663098737597 	loss: 0.4249734878540039
total_loss: 0.4836148525476456 	loss: 0.44027653336524963
total_loss: 0.4821318369656801 	loss: 0.5063344240188599
total_loss: 0.48721315510571 	loss: 0.5011013150215149
total_loss: 0.49015149018168447 	loss: 0.5749050378799438
total_loss: 0.48404801481962206 	loss: 0.4710829257965088
total_loss: 0.4828501836359501 	loss: 0.4063016176223755
total_loss: 0.4852467731684446 	loss: 0.4681253135204315
total_loss: 0.4808730574250221 	loss: 0.5472741723060608
total_loss: 0.48207237070798875 	loss: 0.5661131739616394
total_loss: 0.48313605938851834 	loss: 0.46917298436164856
total_loss: 0.48353707244247196 	loss: 0.6322169899940491
total_loss: 0.48180945399403574 	loss: 0.6094982624053955
total_loss: 0.4829494848400354 	loss: 0.5726996064186096
total_loss: 0.48219003023952245 	loss: 0.445222407579422
total_loss: 0.4848606969565153 	loss: 0.369366317987442
total_loss: 0.4835447753

total_loss: 0.47986665517091753 	loss: 0.5408532619476318
total_loss: 0.48083465968072414 	loss: 0.5566995739936829
total_loss: 0.4813999033719301 	loss: 0.4968506395816803
total_loss: 0.4832864858955145 	loss: 0.5377699136734009
total_loss: 0.4823416720479727 	loss: 0.47774866223335266
total_loss: 0.48008603329211474 	loss: 0.4571529030799866
total_loss: 0.4796696478575468 	loss: 0.46736547350883484
total_loss: 0.4796367326751351 	loss: 0.5150888562202454
total_loss: 0.48168003974854945 	loss: 0.45245254039764404
total_loss: 0.4784090336933732 	loss: 0.5005772113800049
total_loss: 0.4787908728569746 	loss: 0.39607369899749756
total_loss: 0.48272960537672044 	loss: 0.4175424873828888
total_loss: 0.47925040071457625 	loss: 0.520507276058197
total_loss: 0.47860071281343697 	loss: 0.46425843238830566
total_loss: 0.480842418089509 	loss: 0.4816843569278717
total_loss: 0.48034505628049373 	loss: 0.3712867498397827
total_loss: 0.48132050098478796 	loss: 0.34175917506217957
total_loss: 0.4794

### 预测

In [79]:
model.to(device)

DNN(
  (fc1): Linear(in_features=40, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=128, bias=True)
  (fc5): Linear(in_features=128, out_features=10, bias=True)
)

In [80]:
# utt='SV0255_2_00_F0021'
# utt="SV0255_2_07_S1082"
utt=list(phone_label.keys())[110]

# label_list = list(phone_label_dev[utt])
pred_label = model(torch.Tensor(feats[utt]).to(device))
_, answer = torch.max(pred_label.data, 1)
answer_list=list(answer.to("cpu", torch.int).numpy())

In [81]:
utt="SV0287_6_12_S2850"

# label_list = list(phone_label_dev[utt])
pred_label = model(torch.Tensor(feats[utt]).to(device))
_, answer = torch.max(pred_label.data, 1)
answer_list=list(answer.to("cpu", torch.int).numpy())

## 将预测结果映射

In [66]:
decoder = Decoder(Data_show.phone2class)

In [67]:
decoder.show_result(decoder.decode(torch.nn.Softmax()(pred_label)))

/home1/meichaoyang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


	 sil	 nsn	 n	 i2	 h	 ao3	 m	 i3	 ii	 ia3	other
1 :	0.00 	-0.20 	-0.20 	-0.20 	-0.19 	-0.19 	-0.20 	-0.20 	-0.19 	-0.20 	
2 :	0.00 	-0.20 	-0.40 	-0.40 	-0.38 	-0.37 	-0.39 	-0.40 	-0.39 	-0.39 	
3 :	0.00 	-0.20 	-0.40 	-0.60 	-0.57 	-0.57 	-0.57 	-0.59 	-0.59 	-0.59 	
4 :	0.00 	-0.20 	-0.40 	-0.60 	-0.77 	-0.76 	-0.77 	-0.77 	-0.78 	-0.79 	
5 :	0.00 	-0.20 	-0.40 	-0.60 	-0.80 	-0.96 	-0.96 	-0.97 	-0.97 	-0.98 	
6 :	0.00 	-0.20 	-0.40 	-0.60 	-0.80 	-0.99 	-1.16 	-1.16 	-1.16 	-1.17 	
7 :	0.00 	-0.20 	-0.40 	-0.60 	-0.80 	-0.99 	-1.19 	-1.36 	-1.30 	-1.36 	
8 :	0.00 	-0.20 	-0.40 	-0.60 	-0.80 	-0.98 	-1.19 	-1.39 	-1.50 	-1.50 	
9 :	0.00 	-0.20 	-0.40 	-0.60 	-0.77 	-0.97 	-1.18 	-1.39 	-1.58 	-1.70 	
10 :	0.00 	-0.20 	-0.40 	-0.60 	-0.79 	-0.97 	-1.17 	-1.38 	-1.59 	-1.78 	
11 :	0.00 	-0.20 	-0.40 	-0.60 	-0.65 	-0.98 	-1.16 	-1.37 	-1.58 	-1.79 	
12 :	0.00 	-0.20 	-0.40 	-0.60 	-0.77 	-0.84 	-1.18 	-1.36 	-1.56 	-1.78 	
13 :	0.00 	-0.20 	-0.40 	-0.60 	-0.77 	-0.96 	-1.04 	-1.38 	-

In [82]:
Data_show().show_softmax(torch.nn.Softmax()(pred_label))

/home1/meichaoyang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


	 sil	 nsn	 n	 i2	 h	 ao3	 m	 i3	 ii	 ia3	other
0 :	0.97 	0.00 	0.00 	0.00 	0.01 	0.01 	0.00 	0.00 	0.01 	0.00 	
1 :	0.96 	0.00 	0.00 	0.00 	0.01 	0.02 	0.00 	0.00 	0.00 	0.00 	
2 :	0.99 	0.00 	0.00 	0.00 	0.01 	0.01 	0.00 	0.00 	0.00 	0.00 	
3 :	0.99 	0.00 	0.00 	0.00 	0.00 	0.01 	0.00 	0.00 	0.00 	0.00 	
4 :	0.99 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	
5 :	0.98 	0.00 	0.00 	0.00 	0.00 	0.01 	0.00 	0.00 	0.01 	0.00 	
6 :	0.93 	0.00 	0.00 	0.00 	0.00 	0.01 	0.00 	0.00 	0.06 	0.00 	
7 :	0.98 	0.00 	0.00 	0.00 	0.00 	0.01 	0.00 	0.00 	0.00 	0.00 	
8 :	0.93 	0.00 	0.00 	0.00 	0.03 	0.03 	0.00 	0.00 	0.01 	0.00 	
9 :	0.98 	0.00 	0.00 	0.00 	0.01 	0.01 	0.00 	0.00 	0.00 	0.00 	
10 :	0.84 	0.00 	0.00 	0.00 	0.15 	0.01 	0.00 	0.00 	0.00 	0.00 	
11 :	0.96 	0.00 	0.00 	0.00 	0.03 	0.01 	0.00 	0.00 	0.00 	0.00 	
12 :	0.97 	0.00 	0.00 	0.00 	0.02 	0.01 	0.00 	0.00 	0.00 	0.00 	
13 :	0.93 	0.00 	0.00 	0.00 	0.06 	0.01 	0.00 	0.00 	0.00 	0.00 	
14 :	0.98 	0.00 	0.00 	0.00 	0.01 	0.0

In [70]:
function_vector(phone_label[utt])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 4,
       4, 4, 4, 4, 4, 4, 0, 0, 5, 5, 5, 5, 5, 5, 6, 7, 7, 8, 8, 8, 8, 8,
       8, 8, 8, 0, 0, 0])

### “你好米雅”测试

In [93]:
feats_miya_test = { u:d for u,d in kaldi_io.read_mat_scp("/home1/meichaoyang/Dataset/feats/SLR85/far_field/train/feats.scp") }

In [94]:
utt_miya_test=list(feats_miya_test.keys())[420]
# utt_aishell="IC0001W0406"

pred_label_miya_test = model(torch.Tensor(feats_miya_test[utt_miya_test]).to(device))
_, answer_miya_test = torch.max(pred_label_miya_test.data, 1)
answer_miya_test_list=list(answer_miya_test.to("cpu", torch.int).numpy())

In [103]:
title, content = Data_show().show_softmax(torch.nn.Softmax()(pred_label_miya_test))

/home1/meichaoyang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


	 sil	 nsn	 n	 i2	 h	 ao3	 m	 i3	 ii	 ia3	other
0 :	0.34 	0.06 	0.01 	0.00 	0.01 	0.09 	0.00 	0.00 	0.46 	0.01 	
1 :	0.40 	0.01 	0.00 	0.00 	0.01 	0.38 	0.00 	0.00 	0.18 	0.00 	
2 :	0.53 	0.01 	0.00 	0.01 	0.13 	0.21 	0.00 	0.00 	0.10 	0.00 	
3 :	0.27 	0.03 	0.03 	0.01 	0.03 	0.05 	0.00 	0.00 	0.58 	0.00 	
4 :	0.33 	0.01 	0.01 	0.01 	0.08 	0.01 	0.00 	0.00 	0.55 	0.00 	
5 :	0.23 	0.02 	0.01 	0.01 	0.01 	0.03 	0.00 	0.00 	0.68 	0.00 	
6 :	0.36 	0.08 	0.02 	0.00 	0.01 	0.15 	0.00 	0.00 	0.38 	0.00 	
7 :	0.29 	0.03 	0.01 	0.01 	0.01 	0.12 	0.00 	0.00 	0.52 	0.00 	
8 :	0.40 	0.13 	0.04 	0.01 	0.01 	0.16 	0.00 	0.00 	0.26 	0.00 	
9 :	0.42 	0.24 	0.03 	0.00 	0.00 	0.26 	0.01 	0.01 	0.03 	0.00 	
10 :	0.59 	0.01 	0.00 	0.00 	0.02 	0.04 	0.00 	0.01 	0.32 	0.00 	
11 :	0.66 	0.02 	0.00 	0.00 	0.01 	0.08 	0.00 	0.01 	0.21 	0.00 	
12 :	0.58 	0.11 	0.03 	0.00 	0.01 	0.18 	0.00 	0.01 	0.05 	0.02 	
13 :	0.68 	0.04 	0.02 	0.00 	0.03 	0.03 	0.01 	0.01 	0.18 	0.00 	
14 :	0.57 	0.05 	0.00 	0.01 	0.06 	0.0

131 :	0.09 	0.81 	0.00 	0.00 	0.00 	0.01 	0.03 	0.04 	0.00 	0.01 	
132 :	0.07 	0.11 	0.01 	0.00 	0.00 	0.07 	0.06 	0.05 	0.00 	0.62 	
133 :	0.10 	0.14 	0.00 	0.00 	0.00 	0.22 	0.14 	0.09 	0.00 	0.31 	
134 :	0.11 	0.32 	0.00 	0.00 	0.00 	0.09 	0.11 	0.09 	0.00 	0.28 	
135 :	0.02 	0.16 	0.02 	0.00 	0.00 	0.02 	0.03 	0.03 	0.00 	0.72 	
136 :	0.03 	0.16 	0.02 	0.00 	0.00 	0.01 	0.03 	0.04 	0.00 	0.71 	
137 :	0.01 	0.02 	0.01 	0.01 	0.00 	0.01 	0.01 	0.01 	0.00 	0.91 	
138 :	0.00 	0.00 	0.01 	0.05 	0.00 	0.01 	0.01 	0.00 	0.00 	0.91 	
139 :	0.02 	0.04 	0.01 	0.01 	0.00 	0.05 	0.05 	0.01 	0.00 	0.80 	
140 :	0.02 	0.05 	0.01 	0.00 	0.00 	0.04 	0.02 	0.01 	0.00 	0.84 	
141 :	0.01 	0.05 	0.01 	0.00 	0.00 	0.01 	0.02 	0.03 	0.00 	0.86 	
142 :	0.02 	0.05 	0.02 	0.00 	0.00 	0.01 	0.03 	0.03 	0.00 	0.83 	
143 :	0.08 	0.17 	0.00 	0.00 	0.00 	0.03 	0.03 	0.03 	0.00 	0.65 	
144 :	0.08 	0.10 	0.00 	0.00 	0.00 	0.11 	0.48 	0.12 	0.00 	0.10 	
145 :	0.05 	0.04 	0.01 	0.00 	0.00 	0.14 	0.62 	0.05 	0.00 	0.

In [105]:
print(title)
print(content)

	sil	nsn	n	i2	h	ao3	m	i3	ii	ia3	other

0:	0.34	0.06	0.01	0.00	0.01	0.09	0.00	0.00	0.46	0.01	
1:	0.40	0.01	0.00	0.00	0.01	0.38	0.00	0.00	0.18	0.00	
2:	0.53	0.01	0.00	0.01	0.13	0.21	0.00	0.00	0.10	0.00	
3:	0.27	0.03	0.03	0.01	0.03	0.05	0.00	0.00	0.58	0.00	
4:	0.33	0.01	0.01	0.01	0.08	0.01	0.00	0.00	0.55	0.00	
5:	0.23	0.02	0.01	0.01	0.01	0.03	0.00	0.00	0.68	0.00	
6:	0.36	0.08	0.02	0.00	0.01	0.15	0.00	0.00	0.38	0.00	
7:	0.29	0.03	0.01	0.01	0.01	0.12	0.00	0.00	0.52	0.00	
8:	0.40	0.13	0.04	0.01	0.01	0.16	0.00	0.00	0.26	0.00	
9:	0.42	0.24	0.03	0.00	0.00	0.26	0.01	0.01	0.03	0.00	
10:	0.59	0.01	0.00	0.00	0.02	0.04	0.00	0.01	0.32	0.00	
11:	0.66	0.02	0.00	0.00	0.01	0.08	0.00	0.01	0.21	0.00	
12:	0.58	0.11	0.03	0.00	0.01	0.18	0.00	0.01	0.05	0.02	
13:	0.68	0.04	0.02	0.00	0.03	0.03	0.01	0.01	0.18	0.00	
14:	0.57	0.05	0.00	0.01	0.06	0.03	0.00	0.01	0.26	0.00	
15:	0.44	0.06	0.04	0.02	0.03	0.04	0.00	0.00	0.38	0.00	
16:	0.57	0.08	0.02	0.00	0.01	0.10	0.00	0.00	0.21	0.00	
17:	0.76	0.08	0.00	0.00	0.01	0.05	0.

In [96]:
feats[utt_miya_test]

KeyError: 'SV0002_2_08_S1171'

## 非“你好米雅”测试

In [97]:
feats_aishell = { u:d for u,d in kaldi_io.read_mat_scp("../wake_dnn_miya_only/feats_aishell2_test/feats.scp") }

In [87]:
utt_aishell=list(feats_aishell.keys())[420]
utt_aishell="IC0001W0406"

pred_label_aishell = model(torch.Tensor(feats_aishell[utt_aishell]).to(device))
_, answer_aishell = torch.max(pred_label_aishell.data, 1)
answer_aishell_list=list(answer_aishell.to("cpu", torch.int).numpy())

In [88]:
decoder.show_result(decoder.decode(torch.nn.Softmax()(pred_label_aishell)))

/home1/meichaoyang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


	 sil	 nsn	 n	 i2	 h	 ao3	 m	 i3	 ii	 ia3	other
1 :	0.00 	-0.20 	-0.20 	-0.20 	-0.20 	-0.20 	-0.20 	-0.20 	-0.20 	-0.15 	
2 :	0.00 	-0.20 	-0.40 	-0.39 	-0.40 	-0.40 	-0.40 	-0.39 	-0.40 	-0.31 	
3 :	0.00 	-0.20 	-0.40 	-0.59 	-0.59 	-0.60 	-0.60 	-0.59 	-0.59 	-0.46 	
4 :	0.00 	-0.20 	-0.40 	-0.60 	-0.79 	-0.79 	-0.80 	-0.79 	-0.79 	-0.61 	
5 :	0.00 	-0.20 	-0.40 	-0.60 	-0.80 	-0.99 	-0.99 	-0.98 	-0.99 	-0.75 	
6 :	0.00 	-0.20 	-0.40 	-0.60 	-0.80 	-1.00 	-1.19 	-1.18 	-1.18 	-0.91 	
7 :	0.00 	-0.20 	-0.40 	-0.60 	-0.80 	-1.00 	-1.20 	-1.37 	-1.38 	-1.06 	
8 :	0.00 	-0.20 	-0.40 	-0.60 	-0.80 	-1.00 	-1.20 	-1.39 	-1.57 	-1.20 	
9 :	0.00 	-0.20 	-0.40 	-0.60 	-0.80 	-1.00 	-1.20 	-1.39 	-1.59 	-1.36 	
10 :	0.00 	-0.20 	-0.40 	-0.60 	-0.80 	-1.00 	-1.20 	-1.39 	-1.59 	-1.51 	
11 :	0.00 	-0.20 	-0.40 	-0.60 	-0.80 	-1.00 	-1.20 	-1.39 	-1.59 	-1.66 	
12 :	0.00 	-0.20 	-0.40 	-0.60 	-0.80 	-1.00 	-1.20 	-1.39 	-1.59 	-1.74 	
13 :	0.00 	-0.20 	-0.40 	-0.60 	-0.80 	-1.00 	-1.20 	-1.39 	-

131 :	0.00 	-0.20 	-0.40 	-0.39 	-0.68 	-0.96 	-1.14 	-1.29 	-1.44 	51.62 	
132 :	0.00 	-0.20 	-0.40 	-0.55 	-0.59 	-0.88 	-1.16 	-1.34 	-1.49 	52.36 	
133 :	0.00 	-0.20 	-0.40 	-0.55 	-0.74 	-0.78 	-1.08 	-1.36 	-1.54 	53.09 	
134 :	0.00 	-0.20 	-0.40 	-0.54 	-0.74 	-0.94 	-0.98 	-1.27 	-1.56 	53.80 	
135 :	0.00 	-0.20 	-0.40 	-0.57 	-0.73 	-0.93 	-1.14 	-1.18 	-1.47 	54.55 	
136 :	0.00 	-0.20 	-0.40 	-0.58 	-0.76 	-0.93 	-1.13 	-1.34 	-1.38 	55.31 	
137 :	0.00 	-0.20 	-0.40 	-0.59 	-0.77 	-0.95 	-1.13 	-1.33 	-1.54 	56.09 	
138 :	0.00 	-0.20 	-0.40 	-0.57 	-0.78 	-0.97 	-1.15 	-1.33 	-1.53 	56.85 	
139 :	0.00 	-0.20 	-0.40 	-0.59 	-0.77 	-0.98 	-1.17 	-1.35 	-1.53 	57.63 	
140 :	0.00 	-0.20 	-0.40 	-0.59 	-0.79 	-0.97 	-1.18 	-1.37 	-1.55 	58.41 	
141 :	0.00 	-0.20 	-0.40 	-0.59 	-0.79 	-0.98 	-1.17 	-1.38 	-1.57 	59.19 	
142 :	0.00 	-0.19 	-0.40 	-0.59 	-0.79 	-0.99 	-1.18 	-1.36 	-1.58 	59.96 	
143 :	0.00 	-0.19 	-0.39 	-0.59 	-0.79 	-0.98 	-1.19 	-1.37 	-1.56 	60.72 	
144 :	0.00 	

264 :	0.00 	-0.20 	-0.39 	-0.53 	-0.73 	-0.91 	-1.07 	-1.22 	-1.27 	145.44 	
265 :	0.00 	-0.20 	-0.39 	-0.51 	-0.73 	-0.93 	-1.11 	-1.27 	-1.42 	146.15 	
266 :	0.00 	-0.20 	-0.40 	-0.58 	-0.71 	-0.93 	-1.13 	-1.31 	-1.47 	146.93 	
267 :	0.00 	-0.20 	-0.40 	-0.57 	-0.78 	-0.91 	-1.13 	-1.33 	-1.51 	147.68 	
268 :	0.00 	-0.20 	-0.40 	-0.55 	-0.76 	-0.98 	-1.11 	-1.33 	-1.53 	148.40 	
269 :	0.00 	-0.19 	-0.40 	-0.57 	-0.74 	-0.95 	-1.18 	-1.31 	-1.53 	149.14 	
270 :	0.00 	-0.20 	-0.39 	-0.52 	-0.77 	-0.94 	-1.15 	-1.38 	-1.51 	149.85 	
271 :	0.00 	-0.20 	-0.40 	-0.55 	-0.72 	-0.97 	-1.14 	-1.35 	-1.58 	150.58 	
272 :	0.00 	-0.20 	-0.40 	-0.59 	-0.74 	-0.92 	-1.17 	-1.34 	-1.55 	151.36 	
273 :	0.00 	-0.20 	-0.40 	-0.60 	-0.79 	-0.94 	-1.12 	-1.36 	-1.54 	152.14 	
274 :	0.00 	-0.20 	-0.40 	-0.60 	-0.80 	-0.99 	-1.14 	-1.29 	-1.56 	152.90 	
275 :	0.00 	-0.20 	-0.39 	-0.60 	-0.80 	-1.00 	-1.19 	-1.29 	-1.49 	153.64 	
276 :	0.00 	-0.20 	-0.40 	-0.59 	-0.80 	-1.00 	-1.20 	-1.34 	-1.49 	154.38 	

397 :	0.00 	-0.20 	-0.37 	-0.56 	-0.72 	-0.95 	-1.18 	-1.08 	-1.32 	243.08 	
398 :	0.00 	-0.20 	-0.38 	-0.57 	-0.76 	-0.92 	-1.15 	-0.85 	-1.28 	243.43 	
399 :	0.00 	-0.20 	-0.38 	-0.58 	-0.77 	-0.96 	-1.11 	-0.73 	-1.05 	243.86 	
400 :	0.00 	-0.20 	-0.38 	-0.58 	-0.78 	-0.97 	-1.15 	-0.46 	-0.93 	244.17 	
401 :	0.00 	-0.20 	-0.39 	-0.58 	-0.78 	-0.98 	-1.17 	-0.18 	-0.66 	244.46 	
402 :	0.00 	-0.20 	-0.38 	-0.59 	-0.78 	-0.98 	-1.18 	-0.01 	-0.38 	244.86 	
403 :	0.00 	-0.20 	-0.38 	-0.58 	-0.79 	-0.98 	-1.17 	0.01 	-0.21 	245.41 	
404 :	0.00 	-0.20 	-0.38 	-0.58 	-0.78 	-0.99 	-1.18 	0.02 	-0.19 	245.97 	
405 :	0.00 	-0.20 	-0.38 	-0.58 	-0.78 	-0.98 	-1.18 	0.10 	-0.18 	246.46 	
406 :	0.00 	-0.20 	-0.38 	-0.58 	-0.78 	-0.98 	-1.18 	0.08 	-0.10 	247.04 	
407 :	0.00 	-0.20 	-0.39 	-0.58 	-0.78 	-0.98 	-1.18 	0.10 	-0.12 	247.60 	
408 :	0.00 	-0.20 	-0.39 	-0.59 	-0.78 	-0.98 	-1.18 	-0.08 	-0.10 	248.36 	
409 :	0.00 	-0.20 	-0.40 	-0.59 	-0.79 	-0.98 	-1.18 	-0.28 	-0.28 	249.15 	
410 

530 :	0.00 	-0.20 	-0.40 	-0.60 	-0.80 	-1.00 	-1.20 	-1.39 	-1.59 	307.57 	
531 :	0.00 	-0.20 	-0.40 	-0.60 	-0.80 	-1.00 	-1.20 	-1.39 	-1.59 	307.42 	
532 :	0.00 	-0.20 	-0.40 	-0.60 	-0.80 	-1.00 	-1.20 	-1.39 	-1.59 	307.26 	
533 :	0.00 	-0.20 	-0.40 	-0.60 	-0.80 	-1.00 	-1.20 	-1.39 	-1.59 	307.11 	
534 :	0.00 	-0.20 	-0.40 	-0.60 	-0.80 	-1.00 	-1.20 	-1.39 	-1.59 	306.96 	
535 :	0.00 	-0.20 	-0.40 	-0.60 	-0.80 	-1.00 	-1.20 	-1.39 	-1.59 	306.81 	
536 :	0.00 	-0.20 	-0.40 	-0.60 	-0.80 	-1.00 	-1.20 	-1.39 	-1.59 	306.65 	
537 :	0.00 	-0.20 	-0.40 	-0.60 	-0.80 	-1.00 	-1.20 	-1.39 	-1.59 	306.51 	
538 :	0.00 	-0.20 	-0.40 	-0.60 	-0.80 	-1.00 	-1.20 	-1.39 	-1.59 	306.36 	
539 :	0.00 	-0.20 	-0.40 	-0.60 	-0.80 	-1.00 	-1.20 	-1.39 	-1.59 	306.20 	
540 :	0.00 	-0.20 	-0.40 	-0.60 	-0.80 	-1.00 	-1.20 	-1.39 	-1.59 	306.05 	
541 :	0.00 	-0.20 	-0.40 	-0.60 	-0.80 	-1.00 	-1.20 	-1.39 	-1.59 	305.90 	
542 :	0.00 	-0.20 	-0.40 	-0.60 	-0.80 	-1.00 	-1.20 	-1.39 	-1.59 	305.75 	

In [55]:
Data_show().show_softmax(torch.nn.Softmax()(pred_label_aishell))

/home1/meichaoyang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


	 sil	 nsn	 n	 i2	 h	 ao3	 m	 i3	 ii	 ia3	other
0 :	0.95 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.05 	
1 :	0.95 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.05 	
2 :	0.95 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.05 	
3 :	0.94 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.05 	
4 :	0.94 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.06 	
5 :	0.95 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.05 	
6 :	0.95 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.05 	
7 :	0.93 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.06 	
8 :	0.95 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.04 	
9 :	0.94 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.05 	
10 :	0.95 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.04 	
11 :	0.95 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.05 	
12 :	0.94 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.05 	
13 :	0.31 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.68 	
14 :	0.62 	0.00 	0.00 	0.00 	0.00 	0.0

306 :	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.02 	0.00 	0.98 	
307 :	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.02 	0.00 	0.97 	
308 :	0.00 	0.01 	0.00 	0.00 	0.00 	0.00 	0.00 	0.03 	0.00 	0.96 	
309 :	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.03 	0.00 	0.96 	
310 :	0.00 	0.01 	0.00 	0.00 	0.00 	0.00 	0.00 	0.01 	0.00 	0.97 	
311 :	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.05 	0.00 	0.94 	
312 :	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.04 	0.00 	0.95 	
313 :	0.01 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.99 	
314 :	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.99 	
315 :	0.01 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.99 	
316 :	0.01 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.98 	
317 :	0.01 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.99 	
318 :	0.01 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.99 	
319 :	0.01 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.99 	
320 :	0.01 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.

431 :	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.16 	0.00 	0.82 	
432 :	0.01 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.13 	0.00 	0.86 	
433 :	0.01 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.12 	0.00 	0.86 	
434 :	0.01 	0.01 	0.00 	0.00 	0.00 	0.00 	0.00 	0.09 	0.00 	0.89 	
435 :	0.01 	0.01 	0.00 	0.00 	0.00 	0.00 	0.00 	0.05 	0.00 	0.93 	
436 :	0.01 	0.01 	0.00 	0.00 	0.00 	0.00 	0.00 	0.04 	0.00 	0.95 	
437 :	0.00 	0.01 	0.00 	0.00 	0.00 	0.01 	0.00 	0.05 	0.00 	0.92 	
438 :	0.00 	0.01 	0.01 	0.00 	0.00 	0.01 	0.00 	0.06 	0.00 	0.91 	
439 :	0.01 	0.00 	0.01 	0.00 	0.00 	0.01 	0.01 	0.05 	0.00 	0.91 	
440 :	0.01 	0.00 	0.01 	0.00 	0.00 	0.01 	0.01 	0.04 	0.00 	0.93 	
441 :	0.01 	0.00 	0.02 	0.00 	0.00 	0.01 	0.01 	0.04 	0.00 	0.92 	
442 :	0.01 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.01 	0.00 	0.97 	
443 :	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.00 	0.01 	0.00 	0.98 	
444 :	0.01 	0.00 	0.00 	0.05 	0.00 	0.00 	0.00 	0.01 	0.00 	0.92 	
445 :	0.03 	0.00 	0.00 	0.02 	0.00 	0.00 	0.00 	0.00 	0.00 	0.

In [54]:
utt_aishell

'IC0001W0406'

## 保存模型

In [148]:
torch.save(model.to("cpu"), 'model.pkl')
model1 = torch.load('model.pkl')

/home1/meichaoyang/anaconda3/lib/python3.7/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type DNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [59]:
sm = torch.jit.script(model1)
sm.save("phone_cla_model.pt")

In [6]:
model1 = torch.load('model.pkl')

In [7]:
model1

DNN(
  (fc1): Linear(in_features=40, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=128, bias=True)
  (fc5): Linear(in_features=128, out_features=10, bias=True)
)

In [177]:
map1={1:"1-1",2:"2-1",3:"3-1"}

In [178]:
map2={2:"2-2",3:"3-2",4:"4-2"}

In [180]:
{**map1,**map2}

{1: '1-1', 2: '2-2', 3: '3-2', 4: '4-2'}

In [181]:
{**map2,**map1}

{2: '2-1', 3: '3-1', 4: '4-2', 1: '1-1'}